In [ ]:
# pip install watermark lightgbm plotly cufflinks numpy pandas optuna torch pandas_ta gluonts pandas_datareader

In [ ]:
# pip install -U git+https://github.com/unit8co/darts.git@master

In [ ]:
# pip install pytorch-forecasting==0.10.2

In [ ]:
# 1. magic for inline plot
# 2. magic to print version
# 3. magic so that the notebook will reload external python modules
# 4. magic to enable retina (high resolution) plots
# https://gist.github.com/minrk/3301035
%matplotlib inline
%reload_ext watermark
%config InlineBackend.figure_format='retina'

In [ ]:
%watermark

In [ ]:
# conda install -c conda-forge 'u8darts'

### Library imports

In [ ]:
import warnings
warnings.filterwarnings('ignore')

import os
import darts
import pandas as pd
import numpy as np 
from datetime import datetime
import numpy as np

import plotly
import plotly.express as px
import plotly.graph_objects as go

# pip install matplotlib==3.1.2
import matplotlib
import matplotlib.pyplot as plt

import plotly.offline
import cufflinks as cf
cf.go_offline()
cf.set_config_file(offline=False, world_readable=True)

In [ ]:
import copy
from pathlib import Path
import warnings

import pytorch_lightning as pl
from pytorch_lightning.callbacks import EarlyStopping, LearningRateMonitor
from pytorch_lightning.loggers import TensorBoardLogger

import torch
import torch.nn.functional as F

from pytorch_forecasting import Baseline, TemporalFusionTransformer, TimeSeriesDataSet
from pytorch_forecasting.data import GroupNormalizer
from pytorch_forecasting.metrics import SMAPE, PoissonLoss, QuantileLoss
from pytorch_forecasting.models.temporal_fusion_transformer.tuning import optimize_hyperparameters

In [ ]:
# pip install -U "u8darts[torch]"

In [ ]:
import pytorch_forecasting
pytorch_forecasting.__version__

### Reproducibility

In [ ]:
pl.seed_everything(42)

import random
random.seed(0)

import numpy as np
np.random.seed(0)

import torch
torch.manual_seed(0)

In [ ]:
df_m6 = pd.read_csv("M6_Universe.csv", index_col=0)
df_m6.head(5)

In [ ]:
stocks = df_m6[df_m6["class"]=="Stock"]["symbol"].values
etfs = df_m6[df_m6["class"]=="ETF"]["symbol"].values

In [ ]:
SAMPLE_SIZE = 100
FORECAST_HORIZON = 20 #days
PERIODS = 20

In [ ]:
# import numpy as np
# import pandas as pd
# import yfinance as yf
# import warnings

# warnings.filterwarnings("ignore")
# pd.options.display.float_format = '{:.4%}'.format

# # Date range
# start = '2020-01-01'
# end = '2022-04-30'

# # Tickers of assets
# df_m6 = pd.read_csv("M6_Universe.csv", index_col=0)
# df_m6.head(5)
# assets = list(df_m6["symbol"].values)

# # Downloading data
# data = yf.download(assets, start = start, end = end)
# data = data.loc[:,('Adj Close', slice(None))]
# data.columns = assets

In [ ]:
from src.io import get_m6_tickers_data
tickers_data = get_m6_tickers_data(tickers=df_m6["symbol"].to_list(), 
                                   from_date=pd.to_datetime("2018-01-01"))

In [ ]:
from src.ticket_features import calculate_pct_returns, calculate_log_returns, calculate_cum_log_returns, calculate_cum_pct_returns

df = pd.DataFrame.from_dict({k: v['Adj Close'] for k, v in tickers_data.items()})
df_cum_log_returns = df.apply(calculate_cum_log_returns, periods=PERIODS, axis=0)
df_cum_prt_returns = df.apply(calculate_cum_pct_returns, periods=PERIODS, axis=0)
df_log_returns = df.apply(calculate_log_returns, periods=PERIODS, axis=0)
df_prc_returns = df.apply(calculate_pct_returns, periods=PERIODS, axis=0)

In [ ]:
# df_etfs_prc_returns = df[etfs].copy()
# df_stock_prc_returns = df[stocks].copy()

### Reindex dates and fill in with previous values 

In [ ]:
from gluonts.time_feature.holiday import (
    squared_exponential_kernel,
    SpecialDateFeatureSet,
    NEW_YEARS_DAY,
    MARTIN_LUTHER_KING_DAY,
    PRESIDENTS_DAY,
    GOOD_FRIDAY,
    MEMORIAL_DAY,
    INDEPENDENCE_DAY,
    LABOR_DAY,
    THANKSGIVING,
    CHRISTMAS_DAY,
    SUPERBOWL,
    CHRISTMAS_EVE,
    EASTER_SUNDAY,
    EASTER_MONDAY,
    MOTHERS_DAY,
    COLUMBUS_DAY,
    NEW_YEARS_EVE,
    BLACK_FRIDAY,
    CYBER_MONDAY
)

# Example use for using a squared exponential kernel:
kernel = squared_exponential_kernel(alpha=1.0)
sfs = SpecialDateFeatureSet([NEW_YEARS_DAY,
                             MARTIN_LUTHER_KING_DAY,
                             PRESIDENTS_DAY,
                             GOOD_FRIDAY,
                             MEMORIAL_DAY,
                             INDEPENDENCE_DAY,
                             LABOR_DAY,
                             THANKSGIVING,
                             CHRISTMAS_DAY],
                            kernel)

sfs2 = SpecialDateFeatureSet([SUPERBOWL,
                              CHRISTMAS_EVE,
                              EASTER_SUNDAY,
                              EASTER_MONDAY,
                              MOTHERS_DAY,
                              COLUMBUS_DAY,
                              NEW_YEARS_EVE,
                              BLACK_FRIDAY,
                              CYBER_MONDAY],
                            kernel)

In [ ]:
from src.strategy import CustomStrategy

In [ ]:
# Make a pipeline with the steps
from sklearn.pipeline import Pipeline, FeatureUnion, make_pipeline
from transformers import DateTimeTransformer, periodic_spline_transformer
from reduce_memory import ReduceMemoryTransformer
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import MinMaxScaler, StandardScaler

date_time_transforms = make_pipeline(
    DateTimeTransformer()
)

memory_transforms = make_pipeline(
    ReduceMemoryTransformer()
)

In [ ]:
df_stock_returns_quantiles = reindex_weekdays(df.copy())
df_stock_returns_quantiles = (df_stock_returns_quantiles
                              .apply(calculate_pct_returns, periods=PERIODS, axis=0)
                              .apply(lambda x: x + np.random.normal(0, 1e-12, size=(100)), axis=1)
                              .dropna()
                              .rank(1, ascending=True, method='min') // (20.+1e-12) + 1).clip(upper=5).astype(int)
df_stock_returns_quantiles -= 1

In [ ]:
# df_stock_returns_quantiles["2022-04-29":"2022-04-29"].T.reset_index().values

In [ ]:
# df_etf_returns_quantiles = reindex_weekdays(df[etfs]).copy()
# df_etf_returns_quantiles = (df_etf_returns_quantiles
#                               .apply(calculate_pct_returns, periods=PERIODS, axis=0)
#                               .apply(lambda x: x + np.random.normal(0, .000001, size=(50)), axis=1)
#                               .dropna()
#                               .rank(1, ascending=True, method='min') // 10.00000001 + 1).clip(upper=5).astype(int)
# df_etf_returns_quantiles -= 1

In [ ]:
# covariates = get_datetime_covariates(start_index, end_index, memory_transforms, date_time_transforms)

In [ ]:
from src.ticket_features import upper_shadow, lower_shadow, upper_shadow_percent, lower_shadow_percent

tickers_data_enriched = {}

for k, v in tickers_data.items():
    df = v.copy()
    df = reindex_weekdays(df)
    df.ta.strategy(CustomStrategy)
    df.ta.percent_return(cumulative=False, append=True)
    df.ta.percent_return(cumulative=False, length=PERIODS, append=True)
    #     df = (df
    #         .reindex(pd.date_range(start=df.index[0], end=end_index, freq='D'))
    #         .fillna(method='ffill')
    #         .fillna(method='bfill')
    #     )
    df[f"cum_log_returns_{PERIODS}"] = df[["Adj Close"]].apply(calculate_cum_log_returns, periods=PERIODS, axis=0).values
    df[f"log_returns_{PERIODS}"] = df[["Adj Close"]].apply(calculate_log_returns, periods=PERIODS, axis=0).values
    df['high2low'] = df['High'] / df['Low']
    df['var'] = df['Adj Close'].var()
    df['target_var'] = df[f'PCTRET_{PERIODS}'].var()
    df['upper_shadow'] = upper_shadow(df)
    df['lower_shadow'] = lower_shadow(df)
    df['upper_shadow_percent'] = upper_shadow_percent(df)
    df['lower_shadow_percent'] = lower_shadow_percent(df)    
    
    df["GICS_sector/ETF_type"] = df_m6[df_m6["symbol"]==k]["GICS_sector/ETF_type"].values[0]
    df["GICS_industry/ETF_subtype"] = df_m6[df_m6["symbol"]==k]["GICS_industry/ETF_subtype"].values[0]
    df["group"] = k
    df["ticket"] = "stock" if k in stocks else "etf"
    #df["month"] = df.index.month #.astype(str).astype("category")  # categories have be strings
    df["day_of_week"] = df.index.day_of_week #.astype(str).astype("category")  # categories have be strings
    df["log_volume"] = np.log(df["Volume"] + 1e-8)
    df = memory_transforms.fit_transform(df)
    #     scaler = MinMaxScaler() #StandardScaler()
    #     df_scaled = pd.DataFrame(data=scaler.fit_transform(df), 
    #                              index=df.index,
    #                              columns=df.columns)
    #     df_scaled.dropna(inplace=True)
    #     tickers_data_enriched[k] = df_scaled
    tickers_data_enriched[k] = df#[df_stock_returns_quantiles.index[0]:]

In [ ]:
data1 = pd.concat([pd.concat([df_stock_returns_quantiles[[k]].rename(columns={k: "target"}),
                              #pd.concat([df_stock_returns_quantiles[[k]].shift(i).rename(columns={k: f"target_shift_{i}"}) for i in range(1, FORECAST_HORIZON+1)], axis=1).fillna(method="bfill"),
                              tickers_data_enriched[k], 
                             #covariates
                            ], axis=1).dropna().reset_index(drop=True).reset_index() 
                  for k in tickers_data_enriched.keys()])

# data2 = pd.concat([pd.concat([df_etf_returns_quantiles[[k]].rename(columns={k: "target"}), 
#                               #pd.concat([df_etf_returns_quantiles[[k]].shift(i).rename(columns={k: f"target_shift_{i}"}) for i in range(1, FORECAST_HORIZON+1)], axis=1).fillna(method="bfill"),
#                               tickers_data_enriched[k], 
#                              #covariates
#                             ], axis=1).dropna().reset_index(drop=True).reset_index() 
#                   for k in etfs])

# data = pd.concat([data1, data2]).reset_index(drop=True).rename(columns={"index":"time_index"})
data = data1.reset_index(drop=True).rename(columns={"index":"time_index"})

In [ ]:
columns = data.columns[data.isna().any()]
# data.loc[:, columns] = data.loc[:, columns].astype(str).fillna(method='bfill').astype(float)
# data.drop(columns, axis=1)
columns

In [ ]:
time_varying_known_categoricals = ['day_of_week'] # 'month', 
static_columns = ["group", "ticket", 'GICS_sector/ETF_type','GICS_industry/ETF_subtype']
time_var_reals = list(tickers_data_enriched["ABBV"].columns[~tickers_data_enriched["ABBV"].columns.isin(static_columns+time_varying_known_categoricals)]) \
                 #+ list(covariates.reset_index().columns)
data.columns = [d.replace('.','_') for d in data.columns]
time_var_reals = [d.replace('.','_') for d in time_var_reals]
time_var_reals = [i for i in time_var_reals if i!="time_index"]

data = memory_transforms.fit_transform(data)
data['target'] = data['target'].astype(int)
# data['month'] = data['month'].astype(str)
data['day_of_week'] = data['day_of_week'].astype(str)

In [ ]:
data['time_index'].max()

In [ ]:
from datetime import timedelta
from pytorch_forecasting.data.encoders import EncoderNormalizer, TorchNormalizer

max_prediction_length = FORECAST_HORIZON
max_encoder_length = PERIODS
training_cutoff = data['time_index'].max() - 4*FORECAST_HORIZON
# training_cutoff = data['time_index'].min() + 5*FORECAST_HORIZON

training = TimeSeriesDataSet(
    data[lambda x: x['time_index'] <= training_cutoff],
    time_idx="time_index",
    target="target",
    group_ids=["group"],
    min_encoder_length=max_encoder_length // 2,  # keep encoder length long (as it is in the validation set)
    max_encoder_length=max_encoder_length,
    min_prediction_length=1,
    max_prediction_length=max_prediction_length,
    static_categoricals=static_columns,
    static_reals=[], #"avg_population_2017", "avg_yearly_household_income_2017"],
    time_varying_known_categoricals=time_varying_known_categoricals,#"special_days", "month"],
    variable_groups={}, # "special_days": special_days},  # group of categorical variables can be treated as one variable
    time_varying_known_reals=["time_index"],
    time_varying_unknown_categoricals=[],
    time_varying_unknown_reals=["target"]+time_var_reals,
    # lags={"target": [i for i in range(1,max_prediction_length+1)]},
    #target_normalizer=NaNLabelEncoder(),
    #GroupNormalizer(
    #   groups=group_columns, transformation=(F.one_hot, torch.argmax)
    #),  # use softplus and normalize by group
#     target_normalizer = TorchNormalizer(transformation=(F.one_hot, torch.argmax)), 
    add_relative_time_idx=True,
    add_target_scales=False,
    add_encoder_length=False,
    #allow_missing_timesteps=True,
    #add_nan=True
)

# create validation set (predict=True) which means to predict the last max_prediction_length points in time
# for each series
validation = TimeSeriesDataSet.from_dataset(training, data, predict=True, stop_randomization=True)

# create dataloaders for model
batch_size = 128  # set this between 32 to 128
train_dataloader = training.to_dataloader(train=True, batch_size=batch_size, num_workers=0)
val_dataloader = validation.to_dataloader(train=False, batch_size=batch_size * 10, num_workers=0)

In [ ]:
from pytorch_forecasting.metrics import MultiHorizonMetric
from typing import Dict
# from metrics import torch_rps

def rps_loss(y_pred, target):
        probs = F.softmax(y_pred)
        outcome = F.one_hot(target, num_classes=5)
        loss = torch.mean(((torch.cumsum(probs, dim=-1) - torch.cumsum(outcome, dim=-1))**2).double(), dim=-1, keepdim=True)
        return loss

class RPS(MultiHorizonMetric):
    
    def loss(self, y_pred, target):
        y_pred = F.softmax(y_pred, dim=-1)
        target = F.one_hot(target, num_classes=5)
        #loss = torch_rps(y_pred, target)
        loss = torch.mean(((torch.cumsum(y_pred, dim=-1) - torch.cumsum(target, dim=-1))**2).double(), dim=-1, keepdim=True)
        return loss

    def to_quantiles(self, out: Dict[str, torch.Tensor], quantiles=None):
        return out
    
    def to_prediction(self, y_pred: torch.Tensor) -> torch.Tensor:
        """
        Convert network prediction into a point prediction.

        Args:
            y_pred: prediction output of network

        Returns:
            torch.Tensor: point prediction
        """
        return y_pred

In [ ]:
from pytorch_forecasting import Baseline, DeepAR, TimeSeriesDataSet
# from metrics import RPS, torch_rps
# configure network and trainer
early_stop_callback = EarlyStopping(monitor="val_loss", min_delta=1e-4, patience=10, verbose=False, mode="min")
lr_logger = LearningRateMonitor()  # log the learning rate
logger = TensorBoardLogger("lightning_logs")  # logging results to a tensorboard

# configure network and trainer
trainer = pl.Trainer(
    max_epochs=10,
    gpus=0,
    # clipping gradients is a hyperparameter and important to prevent divergance
    # of the gradient for recurrent neural networks
    gradient_clip_val=0.1,
    #limit_train_batches=30,  # coment in for training, running valiation every 30 batches
    # fast_dev_run=True,  # comment in to check that networkor dataset has no serious bugs
    callbacks=[early_stop_callback], #lr_logger, 
    #logger=logger,
)

net = TemporalFusionTransformer.from_dataset(
    training,
    # not meaningful for finding the learning rate but otherwise very important
    learning_rate=0.01,
    hidden_size=16,  # most important hyperparameter apart from learning rate
    # number of attention heads. Set to up to 4 for large datasets
    attention_head_size=1,
    dropout=0.5,  # between 0.1 and 0.3 are good values
    hidden_continuous_size=8,  # set to <= hidden_size
    output_size=5, # 5 categories by default
    loss=RPS(),
    # reduce learning rate if no improvement in validation loss after x epochs
    reduce_on_plateau_patience=4,
    #log_interval=10,  # uncomment for learning rate finder and otherwise, e.g. to 10 for logging every 10 batches
    logging_metrics=torch.nn.ModuleList([RPS()])
)
print(f"Number of parameters in network: {net.size()/1e3:.1f}k")

In [ ]:
# # find optimal learning rate
# res = trainer.tuner.lr_find(
#     net,
#     train_dataloaders=train_dataloader,
#     val_dataloaders=val_dataloader,
#     max_lr=10.0,
#     min_lr=1e-6,
# )

# print(f"suggested learning rate: {res.suggestion()}")
# fig = res.plot(show=True, suggest=True)
# fig.show()

### Baseline

In [ ]:
def rps_loss(y_pred, target):
        probs = F.softmax(y_pred)
        outcome = F.one_hot(target, num_classes=5)
        loss = torch.mean(((torch.cumsum(probs, dim=-1) - torch.cumsum(outcome, dim=-1))**2).double(), dim=-1, keepdim=True)
        return loss
# calculate baseline mean absolute error, i.e. predict next value as the last available value from the history
actuals = torch.cat([y for x, (y, weight) in iter(val_dataloader)])
baseline_predictions = Baseline().predict(val_dataloader)
rps_loss(baseline_predictions, actuals).mean()

In [ ]:
# fit network
trainer.fit(
    net,
    train_dataloaders=train_dataloader,
    val_dataloaders=val_dataloader,
)


### Hyperparameter tuning

In [ ]:
# import pickle

# from pytorch_forecasting.models.temporal_fusion_transformer.tuning import optimize_hyperparameters

# # create study
# study = optimize_hyperparameters(
#     train_dataloader,
#     val_dataloader,
#     model_path="optuna_test",
#     n_trials=100,
#     max_epochs=10,
#     gradient_clip_val_range=(0.01, 1.0),
#     hidden_size_range=(8, 128),
#     hidden_continuous_size_range=(8, 128),
#     attention_head_size_range=(1, 4),
#     learning_rate_range=(0.001, 0.1),
#     dropout_range=(0.1, 0.3),
#     trainer_kwargs=dict(limit_train_batches=30),
#     reduce_on_plateau_patience=4,
#     use_learning_rate_finder=False,  # use Optuna to find ideal learning rate or use in-built learning rate finder
# )

# # save study results - also we can resume tuning at a later point in time
# with open("test_study.pkl", "wb") as fout:
#     pickle.dump(study, fout)

# # show best hyperparameters
# print(study.best_trial.params)


In [ ]:
# load the best model according to the validation loss
# (given that we use early stopping, this is not necessarily the last epoch)
best_model_path = trainer.checkpoint_callback.best_model_path
best_tft = TemporalFusionTransformer.load_from_checkpoint(best_model_path)

In [ ]:
# calcualte mean absolute error on validation set
actuals = torch.cat([y[0] for x, y in iter(val_dataloader)])
x, y = next(iter(val_dataloader))
predictions = best_tft(x)

In [ ]:
# raw predictions are a dictionary from which all kind of information including quantiles can be extracted
raw_predictions, x = best_tft.predict(val_dataloader, mode="raw", return_x=True)

In [ ]:
interpretation = best_tft.interpret_output(raw_predictions, reduction="sum")
best_tft.plot_interpretation(interpretation)

In [ ]:
#F.softmax(raw_predictions['prediction'], dim=-1)#.argmax(dim=-1)
loss = RPS().loss(raw_predictions['prediction'][:,:,:], x['decoder_target'][:,:]).detach().numpy().squeeze()
loss

In [ ]:
raw_predictions['prediction'][:,:,:].shape

In [ ]:
np.mean(loss, axis=0)

In [ ]:

import plotly.express as px

fig = px.imshow(loss.T, 
                color_continuous_scale="Cividis", 
                origin='lower', 
                title=f"Mean loss {np.mean(loss)}"
               )
fig.show()

In [ ]:
loss.shape

In [ ]:
# best_tft.plot_prediction(x, raw_predictions, idx=2, add_loss_to_title=True);

## Submission 

### Predict on new data

In [ ]:
# select last 20 days from data (max_encoder_length is 20)
new_enc_data = []
for ticket in [*stocks, *etfs]:
    ticket_data = data[data.group==ticket]
    encoder_data = ticket_data[lambda x: x.time_index > x.time_index.max() - max_encoder_length]
    new_enc_data.append(encoder_data)
encoder_data = pd.concat(new_enc_data)

# select last known data point and create decoder data from it by repeating it and incrementing the month
# in a real world dataset, we should not just forward fill the covariates but specify them to account
# for changes in special days and prices (which you absolutely should do but we are too lazy here)
new_dec_data = []
for ticket in [*stocks, *etfs]:
    ticket_data = data[data.group==ticket]
    last_ticket_data = ticket_data[lambda x: x.time_index == x.time_index.max()]
    decoder_data = pd.concat(
        [last_ticket_data.assign(time_index=lambda x: x.time_index + i) for i in range(1, max_prediction_length + 1)],
        ignore_index=True,
    )
    new_dec_data.append(decoder_data)
decoder_data = pd.concat(new_dec_data)

#decoder_data["month"] = ((encoder_data["month"]).astype(int) + 1).astype(str).values
decoder_data["day_of_week"] = encoder_data["day_of_week"].values

# combine encoder and decoder data
new_prediction_data = pd.concat([encoder_data, decoder_data], ignore_index=True)

In [ ]:
# raw predictions are a dictionary from which all kind of information including quantiles can be extracted
new_raw_predictions, new_x = best_tft.predict(new_prediction_data, mode="raw", return_x=True)

for idx in range(10):  # plot 10 examples
    best_tft.plot_prediction(new_x, new_raw_predictions, idx=idx, 
                             show_future_observed=False, add_loss_to_title=False);

In [ ]:
def submit_forecasts(raw_predictions, decimals=5):
    df_submission = pd.read_csv("template.csv", index_col=0)
    df_submission.iloc[:,:-1] = F.softmax(raw_predictions['prediction'], dim=-1).numpy()[:,-1,:]
    df_submission.iloc[:,:-1] = df_submission.iloc[:,:-1].round(decimals)
    df_submission.iloc[:, 0] += (1 - df_submission.iloc[:,:-1].sum(1).round(decimals))
    df_submission.round(decimals).to_csv("./results/submission_sub4.csv")

In [ ]:
submit_forecasts(new_raw_predictions)

In [ ]:
df = (pd.read_csv("./results/submission_sub4.csv").iloc[:,1:6] + pd.read_csv("./template.csv").iloc[:,1:6])

In [ ]:
df = pd.read_csv("./results/submission_sub4.csv").iloc[:,1:6] 
df = df/2.

In [ ]:
df1 = pd.read_csv("./results/submission_sub4.csv")
df1.iloc[:,1:6] = df
df1.to_csv("./results/submission_sub4.csv", index=False)

In [ ]:
loss = RPS().loss(raw_predictions['prediction'], x['decoder_target']).detach().numpy().squeeze()

In [ ]:
import seaborn as sns; sns.set_theme()
ax = sns.heatmap(loss.T[:,:])

In [ ]:
plt.plot(loss.T)

In [ ]:
import plotly.express as px

fig = px.imshow(loss.T, 
                color_continuous_scale="Cividis", 
                origin='lower', 
                title="Missing values"
               )
fig.show()

In [ ]:
loss.mean()

In [ ]:
labels = ['Rank 1','Rank 2','Rank 3','Rank 4','Rank 5']